In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from skopt import gp_minimize
from skopt.space import Real
from sklearn.preprocessing import MinMaxScaler

# Create an output directory to store results
output_dir = "Excel_Linear_Results"
os.makedirs(output_dir, exist_ok=True)

# Define the feature column labels for PIMs descriptors
FEATURE_COLUMNS = [
    "C", "H", "O", "N", "F", "S", "System_Size", "a", "b", "c",
    "density", "PLD", "LCD", "N2_SA",
    "Probe_Accessible", "Probe_Occupiable", "Rosenbluth_Weight"
]

# Load and preprocess the PIMs descriptors
data = pd.read_csv("PIM_ExpFeatures.csv")
features = data[FEATURE_COLUMNS].values

# Normalize PIMs descriptors
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

# Load the Qst labels
labels_data = pd.read_csv("PIM_Qst_Labels.csv")
labels = labels_data["Qst_CO2_298K"].values  # Extract Qst labels

# Define hyperparameter search space for Bayesian Optimization (Linear Kernel)
space = [
    Real(1e-6, 1e2, "log-uniform", name="alpha")  # Regularization parameter
]

# Objective function for Bayesian Optimization
def objective(params):
    alpha = params[0]
    testing_r2_scores = []

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, test_idx in kf.split(features):
        X_train, X_test = features[train_idx], features[test_idx]
        y_train, y_test = labels[train_idx], labels[test_idx]

        model = KernelRidge(kernel="linear", alpha=alpha)
        model.fit(X_train, y_train)
        test_predictions = model.predict(X_test)
        testing_r2_scores.append(r2_score(y_test, test_predictions))

    return -np.mean(testing_r2_scores)  # Negative because skopt minimizes

# Run Bayesian Optimization
result = gp_minimize(
    objective,
    space,
    n_calls=500,
    n_initial_points=50,
    random_state=42,
    verbose=True  # Display progress
)

# Extract the best hyperparameters
best_alpha = result.x[0]

# Save best hyperparameters to a CSV file
best_params = pd.DataFrame({"Alpha": [best_alpha]})
best_params.to_csv(os.path.join(output_dir, "Excel_Linear_best_hyperparameters.csv"), index=False)

# Evaluate the best model
kf = KFold(n_splits=5, shuffle=True, random_state=42)
training_r2_scores = []
testing_r2_scores = []
all_train_actuals, all_train_preds = [], []
all_test_actuals, all_test_preds = [], []

for train_idx, test_idx in kf.split(features):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = labels[train_idx], labels[test_idx]

    model = KernelRidge(kernel="linear", alpha=best_alpha)
    model.fit(X_train, y_train)

    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)

    all_train_actuals.extend(y_train)
    all_train_preds.extend(train_predictions)
    all_test_actuals.extend(y_test)
    all_test_preds.extend(test_predictions)

    training_r2_scores.append(r2_score(y_train, train_predictions))
    testing_r2_scores.append(r2_score(y_test, test_predictions))

# Save cross-validation R² scores
#cv_results = pd.DataFrame({
#    "Fold": range(1, len(training_r2_scores) + 1),
#    "Training R^2": training_r2_scores,
#    "Testing R^2": testing_r2_scores
#})
#cv_results.to_csv(os.path.join(output_dir, "Excel_Linear_cv_results.csv"), index=False)

# Compute average R²
average_training_r2 = np.mean(training_r2_scores)
average_testing_r2 = np.mean(testing_r2_scores)

# Generate parity plots
#for dataset, actuals, preds, avg_r2, filename in zip(
#    ["Training", "Testing"],
#    [all_train_actuals, all_test_actuals],
#    [all_train_preds, all_test_preds],
#    [average_training_r2, average_testing_r2],
#    ["Excel_Linear_training_parity_plot.png", "Excel_Linear_testing_parity_plot.png"]
#):
#    plt.figure()
#    plt.scatter(actuals, preds, alpha=0.7, label=f"{dataset} Data")
#    plt.plot([min(actuals), max(actuals)], [min(actuals), max(actuals)], 'k--', label="Perfect Prediction")
#    plt.xlabel("Actual Values")
#    plt.ylabel("Predicted Values")
#    plt.title(f"{dataset} Parity Plot ($R^2 = {avg_r2:.3f}$)")
#    plt.legend()
#    plt.grid()
#    plt.savefig(os.path.join(output_dir, filename))
#    plt.close()

#print(f"Results saved in {output_dir}")


Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0025
Function value obtained: 3.9649
Current minimum: 3.9649
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0027
Function value obtained: -0.9039
Current minimum: -0.9039
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0020
Function value obtained: 3.0266
Current minimum: -0.9039
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0020
Function value obtained: 0.0623
Current minimum: -0.9039
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0020
Function value obtained: 0.1462
Current minimum: -0.9039
Iteration No: 6 started. Evalu

/Users/samusmaster210/.conda/envs/Chem5/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1.53421106546219e-06] before, using random point [0.007306109209078901]
  warnings.warn(


Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 0.5946
Function value obtained: 0.1526
Current minimum: -0.9575
Iteration No: 86 started. Searching for the next optimal point.
Iteration No: 86 ended. Search finished for the next optimal point.
Time taken: 0.5959
Function value obtained: -0.9572
Current minimum: -0.9575
Iteration No: 87 started. Searching for the next optimal point.
Iteration No: 87 ended. Search finished for the next optimal point.
Time taken: 0.5594
Function value obtained: -0.9568
Current minimum: -0.9575
Iteration No: 88 started. Searching for the next optimal point.
Iteration No: 88 ended. Search finished for the next optimal point.
Time taken: 0.5561
Function value obtained: -0.9575
Current minimum: -0.9575
Iteration No: 89 started. Searching for the next optimal point.
Iteration No: 89 ended. Search finished for the next optimal point.
Time taken: 0.7669
Function value obtained: -0.9574
Current minimum: -0.9575
Iteration No: 90 sta

/Users/samusmaster210/.conda/envs/Chem5/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [4.4393213660416315e-06] before, using random point [0.00012906972906201656]
  warnings.warn(


Iteration No: 100 ended. Search finished for the next optimal point.
Time taken: 0.8532
Function value obtained: -0.6062
Current minimum: -0.9575
Iteration No: 101 started. Searching for the next optimal point.
Iteration No: 101 ended. Search finished for the next optimal point.
Time taken: 0.8386
Function value obtained: -0.9573
Current minimum: -0.9575
Iteration No: 102 started. Searching for the next optimal point.
Iteration No: 102 ended. Search finished for the next optimal point.
Time taken: 0.9338
Function value obtained: -0.9572
Current minimum: -0.9575
Iteration No: 103 started. Searching for the next optimal point.
Iteration No: 103 ended. Search finished for the next optimal point.
Time taken: 1.0312
Function value obtained: -0.9528
Current minimum: -0.9575
Iteration No: 104 started. Searching for the next optimal point.
Iteration No: 104 ended. Search finished for the next optimal point.
Time taken: 0.8545
Function value obtained: -0.9571
Current minimum: -0.9575
Iteration 

...

Iteration No: 434 ended. Search finished for the next optimal point.
Time taken: 13.7588
Function value obtained: 0.0684
Current minimum: -0.9575
Iteration No: 435 started. Searching for the next optimal point.
Iteration No: 435 ended. Search finished for the next optimal point.
Time taken: 14.5543
Function value obtained: -0.9575
Current minimum: -0.9575
Iteration No: 436 started. Searching for the next optimal point.
Iteration No: 436 ended. Search finished for the next optimal point.
Time taken: 12.5732
Function value obtained: -0.9569
Current minimum: -0.9575
Iteration No: 437 started. Searching for the next optimal point.
Iteration No: 437 ended. Search finished for the next optimal point.
Time taken: 13.7988
Function value obtained: -0.9540
Current minimum: -0.9575
Iteration No: 438 started. Searching for the next optimal point.
Iteration No: 438 ended. Search finished for the next optimal point.
Time taken: 13.6400
Function value obtained: -0.9575
Current minimum: -0.9575
Iterat

/Users/samusmaster210/.conda/envs/Chem5/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [5.065290006252902e-06] before, using random point [0.0516727919322468]
  warnings.warn(


Iteration No: 446 ended. Search finished for the next optimal point.
Time taken: 13.9884
Function value obtained: 0.0607
Current minimum: -0.9575
Iteration No: 447 started. Searching for the next optimal point.
Iteration No: 447 ended. Search finished for the next optimal point.
Time taken: 14.9410
Function value obtained: -0.9575
Current minimum: -0.9575
Iteration No: 448 started. Searching for the next optimal point.
Iteration No: 448 ended. Search finished for the next optimal point.
Time taken: 13.3650
Function value obtained: -0.9562
Current minimum: -0.9575
Iteration No: 449 started. Searching for the next optimal point.
Iteration No: 449 ended. Search finished for the next optimal point.
Time taken: 14.0993
Function value obtained: -0.9359
Current minimum: -0.9575
Iteration No: 450 started. Searching for the next optimal point.
Iteration No: 450 ended. Search finished for the next optimal point.
Time taken: 14.0782
Function value obtained: -0.9570
Current minimum: -0.9575
Iterat

/Users/samusmaster210/.conda/envs/Chem5/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [3.579441141858258e-06] before, using random point [29.32303227109086]
  warnings.warn(


Iteration No: 468 ended. Search finished for the next optimal point.
Time taken: 14.7288
Function value obtained: 18.7341
Current minimum: -0.9575
Iteration No: 469 started. Searching for the next optimal point.
Iteration No: 469 ended. Search finished for the next optimal point.
Time taken: 15.3383
Function value obtained: -0.9570
Current minimum: -0.9575
Iteration No: 470 started. Searching for the next optimal point.
Iteration No: 470 ended. Search finished for the next optimal point.
Time taken: 15.7636
Function value obtained: -0.9465
Current minimum: -0.9575
Iteration No: 471 started. Searching for the next optimal point.
Iteration No: 471 ended. Search finished for the next optimal point.
Time taken: 16.0243
Function value obtained: -0.9574
Current minimum: -0.9575
Iteration No: 472 started. Searching for the next optimal point.
Iteration No: 472 ended. Search finished for the next optimal point.
Time taken: 15.1812
Function value obtained: -0.9521
Current minimum: -0.9575
Itera

In [2]:
# Final parity plot with both training and testing results
plt.figure(figsize=(6, 6))
plt.rcParams.update({'font.family': 'Times New Roman'})

# Scatter points
plt.scatter(all_train_actuals, all_train_preds, color='blue', alpha=0.7, edgecolor='k', label=f"Training ($R^2$ = {np.mean(training_r2_scores):.3f})")
plt.scatter(all_test_actuals, all_test_preds, color='red', alpha=0.7, edgecolor='k', label=f"Testing ($R^2$ = {np.mean(testing_r2_scores):.3f})")

# Diagonal reference line
min_val = min(min(all_train_actuals), min(all_test_actuals))
max_val = max(max(all_train_actuals), max(all_test_actuals))
plt.plot([min_val, max_val], [min_val, max_val], 'k--', linewidth=1)

# Labels and title
plt.xlabel("Actual Qst", fontsize=14)
plt.ylabel("Predicted Qst", fontsize=14)
plt.title("Linear KRR Prediction Results", fontsize=14)

# Styling
plt.xticks(fontsize=12, color='black')
plt.yticks(fontsize=12, color='black')
plt.grid(True, linestyle='--', linewidth=0.5)
plt.legend(loc='upper left', fontsize=12, frameon=False)
plt.gca().set_aspect('equal', adjustable='box')
plt.tight_layout()

# Save as PDF
plt.savefig(os.path.join(output_dir, "Linear_combined_parity_plot.pdf"), format='pdf')
plt.close()
